In [2]:
#LOAD AND IMPORT MODELS

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = ('C:/Users/HP/Desktop/Skin cancer/Train') # train images dir  
val = ('C:/Users/HP/Desktop/Skin cancer/Test') # test image dir

# labels or classes in both train and test dir
labels = ['actinic keratosis',
 'basal cell carcinoma',
 'dermatofibroma',
 'melanoma',
 'nevus',
 'pigmented benign keratosis',
 'seborrheic keratosis',
 'squamous cell carcinoma',
 'vascular lesion']

In [5]:
os.listdir('C:/Users/HP/Desktop/Skin cancer/Train')

['nevus',
 'actinic keratosis',
 'vascular lesion',
 'seborrheic keratosis',
 'pigmented benign keratosis',
 'melanoma',
 'basal cell carcinoma',
 'dermatofibroma',
 'squamous cell carcinoma']

In [6]:
# load train and test images into list
# Resize images
# Convert images from rgb to greyscale if required

x_train=[]

for folder in os.listdir(train):

    sub_path=train+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)
        
x_test=[]

for folder in os.listdir(val):

    sub_path=val+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)


In [7]:
# list converted to numpy arrays

train_x=np.array(x_train, dtype=np.float32)
test_x=np.array(x_test, dtype=np.float32)

In [8]:
#Normalization done by dividing by the maximum value ie 225

train_X = []
for number in train_x:
    train_X.append(number / 255.0)

test_X = []
for number in test_x:
    test_X.append(number / 255.0)

In [9]:
# Data augmentation using IMAGEGENERATOR

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
training_set = train_datagen.flow_from_directory(train,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(val,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 2317 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


In [11]:
training_set.class_indices
test_set.class_indices
train_y=training_set.classes
test_y=test_set.classes

train_y.shape, test_y.shape

((2317,), (118,))

In [12]:
height = 224
width = 224
channels = 3
input_shape = (height, width, channels)


In [13]:
# USE OF VGG OR EFFICIENTNET PRETRAINED WEIGHTS TO ACHIEVE IMPROVED ACCURACY 
from tensorflow.keras.applications.vgg19 import VGG19
vgg = VGG19(input_shape=input_shape, weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

80150528/80134624 [==============================] - 1s 0us/step


In [14]:
x = Flatten()(vgg.output)
prediction = Dense(9, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
# COMPILE MODEL
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [16]:
# APPLTY EARLY STOPPING
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [17]:
# FIT MODEL TO DATA

model.fit(x=training_set, validation_data=test_set ,epochs=10 ,callbacks=[early_stop],verbose=1,batch_size=32,shuffle=True)

Epoch 1/10
73/73 [==============================] - 1458s 20s/step - loss: 2.0215 - accuracy: 0.3297 - val_loss: 2.2047 - val_accuracy: 0.1525
Epoch 2/10
73/73 [==============================] - 1425s 20s/step - loss: 1.4938 - accuracy: 0.4722 - val_loss: 2.2149 - val_accuracy: 0.3051
Epoch 3/10
73/73 [==============================] - 1403s 19s/step - loss: 1.2406 - accuracy: 0.5568 - val_loss: 2.1833 - val_accuracy: 0.3305
Epoch 4/10
73/73 [==============================] - 1406s 19s/step - loss: 1.1111 - accuracy: 0.6111 - val_loss: 2.2367 - val_accuracy: 0.2712
Epoch 5/10
73/73 [==============================] - 1426s 20s/step - loss: 1.0012 - accuracy: 0.6573 - val_loss: 2.4007 - val_accuracy: 0.3305
Epoch 6/10
73/73 [==============================] - 1423s 19s/step - loss: 0.9386 - accuracy: 0.6659 - val_loss: 1.8658 - val_accuracy: 0.4237
Epoch 7/10
73/73 [==============================] - 1397s 19s/step - loss: 0.8544 - accuracy: 0.6962 - val_loss: 1.8788 - val_accuracy: 0.3475

In [20]:
# SAVE MODEL 
# MODEL ARCHITECTURE SAVED AS JSON AND WEIGHTS AS HDH5

from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("C:/Users/HP/Desktop/Skin cancer/model/model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("C:/Users/HP/Desktop/Skin cancer/model/model.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
###load model for use
from keras.models import model_from_json

# load json and create model
json_file = open('C:/Users/HP/Desktop/Skin cancer/model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("C:/Users/HP/Desktop/Skin cancer/model/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
#load new image and test model

In [ ]:

from PIL import Image 
import cv2
import numpy as np

img = cv2.imread("Insert new image here", cv2.IMREAD_COLOR)
new_image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
x = cv2.resize(new_image,(224,224))
X = x.reshape(1,224,224,3)
X=np.array(X, dtype=np.float32)
X = X/225.0    
X

y = loaded_model.predict(X)
result= np.argmax(y, axis=1) # code returns maximum value from the probability

print(result())
